# SCADA System Testbed for Cybersecurity Research Using Machine Learning Approach.ipynb

In [ ]:
#https://arxiv.org/abs/1904.00753

## Import libraries

In [ ]:
#Data wrangling
# numpy and pandas for data manipulation
import pandas as pd
import numpy as np

import missingno
from collections import Counter
#Data visualization
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
#Split Data
from sklearn.model_selection import train_test_split
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
#Machine learning models
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
#Model evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, f1_score, matthews_corrcoef
#Hyperparameter tuning
from sklearn.model_selection import GridSearchCV
# File system manangement
import os
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
print('!Done')

In [ ]:
#Libraries version
print("Numpy:", np.__version__)
print("Pandas:", pd.__version__)
print("Seaborn:", sns.__version__)

* Numpy: 1.19.5
* Pandas: 1.1.5
* Seaborn: 0.11.1

## Import and read dataset

In [ ]:
data = pd.read_csv('../input/scada-dataset/cyberdata.csv')
print('Data shape: ',data.shape)
data.head()

## Data description

*   Total Packets (TotPkts) Total transaction packet count

*   Total Bytes (TotBytes) Total transaction bytes

*   Source packets (SrcPkts) Source/Destination packet count

*   Destination Packets (DstPkts) Destination/Source packet count

*   Source Bytes (SrcBytes) Source/Destination transaction bytes

*   Source Port (Sport) Port number of the source







# Exploratory Data Analysis (EDA)
Exploratory data analysis is the process of visualising and analysing data to extract insights. In other words, we want to summarise important characteristics and trends in our data in order to gain a better understanding of our dataset.

In [ ]:
# quick look at our data types and null contents
data.info()

In [ ]:
data.describe(include='all')

## Examine Missing Values

In [ ]:
# Function to calculate missing values by column 
def missing_values_table(df):
        #Missin value matrix
        print(missingno.matrix(df))

        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values = missing_values_table(data)
print('     Missin value table:')
missing_values.head(20)

*Great! My dataset don't have missing values*

In [ ]:
#Columns of my dataset
data.columns

In [ ]:
#Categorical variable Target
sns.countplot(data['Target'], palette=['#04009A',"#CF0000"])
plt.title('Normal and Anormalus traffic')
plt.show()

*The amount of data collected for Anomalous Traffic is very less that for Normal traffic, so, my dataset is imbalanced, and that can affect my accuracy, and the detection of anomalous traffic.*

In [ ]:
#I want to see how many samples were collected for each Port 
#Sport, value counts, 
data['Sport'].value_counts().sort_values(ascending = False)


*In total, 24608 different ports, some ports only collected a sample, so the greatest distribution of network traffic is located in some ports.*

In [ ]:
sns.distplot(data['Sport'])
plt.title('Sport Distribution')

*We can how the collected traffic is distributed through the differents ports and where the peaks are located*


In [ ]:
#I create two dataframes to separate where the collected traffic was normal and anomalus to study it separately
data_port_nor = data.loc[data['Target'] == 0]
data_port_ano = data.loc[data['Target'] == 1]
print("DataFrame corresponding to normal traffic")
print(data_port_nor)
print("DataFrame corresponding to anomalus traffic")
print(data_port_ano.reset_index().drop(columns = 'index'))
#for check if is the size of each dataframe 
#if  (data_port_nor.shape[0] + data_port_ano.shape[0] == data.shape[0]):
 # print(True)

In [ ]:
#We can see the the difference between normal and anomalus traffic collected 
print("Normal traffic: "+ str((data_port_nor.shape[0] / data.shape[0]) * 100 ) + "% of my dataset") 
print("Anomalus traffic: "+ str((data_port_ano.shape[0] / data.shape[0]) * 100) + "% of my dataset") 

*We see another measure of how unbalanced my dataset is*

In [ ]:
#I want to see how the normal traffic is distributed through the different ports
sns.distplot(data_port_nor['Sport'])
plt.title('Sport Distribution Normal Traffic')

*Peaks for normal traffic are located between ports 42K and 68K*

In [ ]:
plt.scatter(data_port_nor['Sport'], data_port_nor['Target'])

*Most normal traffic come from ports between 42K and 70K*

In [ ]:
#I want to see how the anomalus traffic is distributed through the different ports
sns.distplot(data_port_ano['Sport'])
plt.title('Sport Distribution Anomalus Traffic ')

*Peaks for anomalus traffic are located around ports 35K, 42K and 52K*

In [ ]:
plt.scatter(data_port_ano['Sport'], data_port_ano['Target'])

*Most anomalus traffic come from ports between 30K and 65K*

In [ ]:
x_serie = data_port_nor['Sport'].value_counts().sort_values(ascending = False)
print(type(x_serie))
df = pd.DataFrame(x_serie)
df = df.loc[df['Sport'] > 1]
df

In [ ]:
#I want to see how the amount of normal traffic is distributed in the ports 
sns.distplot(df['Sport'].value_counts().sort_values(ascending = False))

*The largest number of normal traffic samples has a peak in the lower ports with other peaks distributed in the remaining ports*

In [ ]:
x_serie = data_port_ano['Sport'].value_counts().sort_values(ascending = False)
print(type(x_serie))
df = pd.DataFrame(x_serie)
df = df.loc[df['Sport'] > 1]
df

In [ ]:
#I want to see how the amount of anomalus traffic is distributed in the ports 
sns.distplot(df['Sport'].value_counts().sort_values(ascending = False))

*Most anomalus traffic samples peak at ports less than 3000*

In [ ]:
heat_map = data.corr()
sns.heatmap(heat_map, annot= True, fmt='.2f', cmap = 'coolwarm')

*Some features have a high correlation, further in this notebook I'll try delete some of that features and check what happens with the accuracy of the model after that.*

# Split training data
We need to first split our training data into independent variables or predictor variables, represented by X as well as dependent variable or response variable, represented by Y.

In [ ]:
X = data.drop('Target', axis=1)
y = data['Target']

In [ ]:
#Split Dataset
def split_data(X,y):
  return train_test_split(X, y, test_size = 0.2, random_state = 1)

X_train, x_test, Y_train, y_test = split_data(X, y)


### Ordinary Least Squares

In [ ]:
import statsmodels.api as sm

In [ ]:
X_train, x_test, Y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 1)
X = X_train
y = Y_train

In [ ]:
glm_binom = sm.GLM(y, X, family=sm.families.Binomial())
res = glm_binom.fit()
print(res.summary())

# Fit model to data and make prediction

In [ ]:
#Models
models = []
models.append(('Random Forest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
#models.append(('Naives Bayes', GaussianNB()))
  #Lousy accuracy with Naives Bayes
models.append(('Logistic Regression', LogisticRegression()))

In [ ]:
results = []
names = []
cm = []
f1 = []  
mcc = []
def train_models(X_train, Y_train, x_test, y_test, model):
    
  results = []
  names = []
  cm = []
  f1 = []  
  mcc = []
  #acc_log = []
  
  for name, model in models:
    cv_results = model.fit(X_train, Y_train)
    results.append(cv_results)
    names.append(name)
    #acc = round(model.score(X_train, Y_train) * 100, 2)
    #acc_log.append(acc)
    y_pred = model.predict(x_test)
    c_matrix = confusion_matrix(y_test, y_pred)
    cm.append(c_matrix)
    f1_acc = f1_score(y_test, y_pred)
    f1.append(f1_acc)
    matthews = matthews_corrcoef(y_test, y_pred)
    mcc.append(matthews)
    print(name, ' -> Accuracy f1: ', f1_acc)
    print(name, ' -> Accuracy MCC: ', matthews)
    print('CM ->: ')
    print(c_matrix)


In [ ]:
#call train_models
train_models(X_train, Y_train, x_test, y_test, models)

*Random Forest and Decision Tree show the better results in the confusion matrix, however Logistic Regression shows incorrect predictions, 1936 false predictions for anomalus traffic and 431 false predictions for normal traffic*

# K-Fold cross validation


In [ ]:
k_fold_results = []
cv_mean = []
cv_std = []

In [ ]:
def k_fold(X_train, Y_train):
    for name, classifier in models:
        k_fold_v = cross_val_score(classifier, X_train, Y_train, scoring='f1', cv = 3) #Put 10   
        #https://scikit-learn.org/stable/modules/model_evaluation.html
        k_fold_results.append(k_fold_v)
        cv_mean.append(k_fold_v.mean())
        cv_std.append(k_fold_v.std())
    print('Ready!')



In [ ]:
k_fold(X_train, Y_train) 

In [ ]:
print(models[1][0])

In [ ]:
def show_k_fold_results(models, cv_mean, cv_std): 
    
    algorithms = ['Random Forest', 'Decision Tree', 'Logistic Regression']
    cv_res = pd.DataFrame({'Cross Validation Mean': cv_mean, 'Cross Validation Std': cv_std, 'Algorithm': algorithms})
    cv_res.sort_values(by = 'Cross Validation Mean', ascending = False, ignore_index = True)

    print(cv_res)

    sns.barplot('Cross Validation Mean', 'Algorithm', data = cv_res, order = cv_res.sort_values(by = 'Cross Validation Mean', ascending = False)['Algorithm'], palette = 'Set3', **{'xerr': cv_std})
    plt.ylabel('Algorithm')
    plt.title('Cross Validation Scores')
    plt.show()

In [ ]:
show_k_fold_results(models, cv_mean, cv_std)    

*Random Forest show the better results for Cross Validation*

In [ ]:
print(type(X_train))

In [ ]:
#Data frame to Numpy 
X = X_train.to_numpy()
y = Y_train.to_numpy()

In [ ]:
print(X)
print(y)
ports = X[:,0]
ports = ports[:, None] #2D array
#X = iris.data[:, 2]
#X = X[:, None]
print(ports)
X_train, x_test, Y_train, y_test = split_data(ports, y)

In [ ]:
from mlxtend.plotting import plot_decision_regions 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools

gs = gridspec.GridSpec(2, 2)

fig = plt.figure(figsize=(10,8))

clf1 = RandomForestClassifier()
clf2 = DecisionTreeClassifier()
clf3 = LogisticRegression()


labels = ['Random Forest', 'Decision Tree', 'Logistic Regression']
for clf, lab, grd in zip([clf1, clf2, clf3],
                         labels,
                         itertools.product([0, 1], repeat=2)):

    clf.fit(X_train, Y_train)
    
    y_pred = clf.predict(x_test)
    c_matrix = confusion_matrix(y_test, y_pred)
    cm.append(c_matrix)
    #f1_acc = f1_score(y_test, y_pred)
    #f1.append(f1_acc)
    #matthews = matthews_corrcoef(y_test, y_pred)
    #mcc.append(matthews)
    #print(name, ' -> Accuracy f1: ', f1_acc)
    #print(name, ' -> Accuracy MCC: ', matthews)
    print('CM ->: ')
    print(c_matrix)
    
    ax = plt.subplot(gs[grd[0], grd[1]])
    fig = plot_decision_regions(X_train, Y_train, clf=clf, legend=2)
    plt.title(lab)

plt.show()

*I'll delete some features with high correlation for check the accuracy of my model after that*

In [ ]:
data = data.drop(columns='TotPkts')
data

In [ ]:
heat_map = data.corr()
sns.heatmap(heat_map, annot= True, fmt='.2f', cmap = 'coolwarm')

In [ ]:
k_fold_results = []
cv_mean = []
cv_std = []
X = data.drop('Target', axis=1)
y = data['Target']
X_train, x_test, Y_train, y_test = split_data(X, y)
train_models(X_train, Y_train, x_test, y_test, models)
k_fold(X_train, Y_train)

*After delete the feature 'TotPkts', the accuracy and the confusion matrix is practically the same*

In [ ]:
show_k_fold_results(models, cv_mean, cv_std) 

*Random Forest keep showing the best results for Cross Validation*

*I'll delete the feature 'SrcPkts' and check the accuracy of my model after that*

In [ ]:
data = data.drop(columns='SrcPkts')
data

In [ ]:
heat_map = data.corr()
sns.heatmap(heat_map, annot= True, fmt='.2f', cmap = 'coolwarm')

In [ ]:
k_fold_results = []
cv_mean = []
cv_std = []
X = data.drop('Target', axis=1)
y = data['Target']
X_train, x_test, Y_train, y_test = split_data(X, y)
train_models(X_train, Y_train, x_test, y_test, models)
k_fold(X_train, Y_train)

*After delete the feature 'SrcBytes', the accuracy and the confusion matrix is practically the same*

In [ ]:
show_k_fold_results(models, cv_mean, cv_std)

*Random Forest keep showing the best results for Cross Validation*

*I'll delete the feature 'SrcBytes' and check the accuracy of my model after that*

In [ ]:
data = data.drop(columns='SrcBytes')
data

In [ ]:
heat_map = data.corr()
sns.heatmap(heat_map, annot= True, fmt='.2f', cmap = 'coolwarm')

In [ ]:
k_fold_results = []
cv_mean = []
cv_std = []
X = data.drop('Target', axis=1)
y = data['Target']
X_train, x_test, Y_train, y_test = split_data(X, y)
train_models(X_train, Y_train, x_test, y_test, models)
k_fold(X_train, Y_train)

*After delete the feature 'SrcBytes', the accuracy and the confusion matrix is worse*

In [ ]:
show_k_fold_results(models, cv_mean, cv_std)

#### If I want to simplify my model I can eliminate the characteristics 'TotPkts' and 'SrcPkts', because they have a high correlation and do not provide much new information to my model, but if the complexity of my model is not very important, eliminating these characteristics will not harm the final results much. 